# Evaluation of results from the simulations for the different scenarios
1. Reserved bandwidth for the different services at the different nodes
2. Latencies of the different services at different receivers
3. Queue lengths of the different priorities at the different nodes

In [15]:
import pandas as pd
import numpy as np
import re
import os
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
%matplotlib inline  

In [16]:
# # read the omnetpp csv results into panda data frames.
# reservation = pd.read_csv('reservedBandwidth.csv')
# latencies = pd.read_csv('serviceLatencies.csv')
# queues = pd.read_csv('allQueueLengths.csv')

## Definitions

## Frequently used functions

In [17]:
def parse_if_number(s):
    try: return float(s)
    except: return True if s=="true" else False if s=="false" else s if s else None

def parse_ndarray(s):
    return np.fromstring(s, sep=' ') if s else None

def parseCSV(csvFile):
    return pd.read_csv(csvFile, converters = {
        'attrvalue': parse_if_number,
        'binedges': parse_ndarray,
        'binvalues': parse_ndarray,
        'vectime': parse_ndarray,
        'vecvalue': parse_ndarray})

def moduleIdxIsNaN(data, idx):
    return data['module'][idx] is np.nan

def idxHasVecVal(data, idx):
    return data['vecvalue'][idx] is not None

def getVecValAndTime(data, i, j):
    return data['vecvalue'][i][j], data['vectime'][i][j]

def isModuleSwitchPort(module):
    return '.S[' in module and '.etherMAC[' in module

def isModuleNode(module):
    return '.N[' in module

def isModuleService(module):
    return '.services[' in module

def removeNetworkFromModule(module):
    # remove everything before the first dot
    return module[module.find('.')+1:]

def isStatReservedBandwidth(stat):
    return 'reservedBandwidth' in stat

def isStatServiceLatency(stat):
    return 'rxLatency' in stat

def getRunName(run):
    return run.split('-')[0]

def getParamForModule(data, module, param):
    moduleParamPath = "**." + removeNetworkFromModule(module) + "." + param
    # filter data for 'type' = param
    for i in range(0, len(data)):
        if data['type'][i] == "param" and data['attrname'][i] == moduleParamPath:
            return data['attrvalue'][i]
    return None

def findServiceIdForModule(data, module):
    return int(getParamForModule(data, module, "serviceId"))
    
def parseModuleNameQueue(module):
    if isModuleNode(module):
        nodeNo = int(re.search(r'N\[(.*?)\]', module).group(1))
        cbsPrio = int(re.search(r'queue\[(.*?)\]', module).group(1))
        return "N[{}] Queue[{}]".format(nodeNo, cbsPrio)
    elif isModuleSwitchPort(module):
        switchNo = int(re.search(r'S\[(.*?)\]', module).group(1))
        portNo = int(re.search(r'etherMAC\[(.*?)\]', module).group(1))
        cbsPrio = int(re.search(r'queue\[(.*?)\]', module).group(1))
        return "S[{}] Port[{}] Queue[{}]".format(switchNo, portNo, cbsPrio)
    return None
    
def parseModuleNameSwitchCBS(module):
    # extract switch and algorithm number from SomeIp_Qos_Small.S[0].etherMAC[0].shaper.transmissionSelectionAlgorithm[5] 
    if isModuleSwitchPort(module):
        switchNo = int(re.search(r'S\[(.*?)\]', module).group(1))
        portNo = int(re.search(r'etherMAC\[(.*?)\]', module).group(1))
        cbsPrio = int(re.search(r'Algorithm\[(.*?)\]', module).group(1))
        return "S[{}] Port[{}] Queue[{}] CBS".format(switchNo, portNo, cbsPrio)
    return None

def parseNodeName(module):
    if isModuleNode(module):
        nodeNo = int(re.search(r'N\[(.*?)\]', module).group(1))
        return "N[{}]".format(nodeNo)

def parseServiceName(data, module):
    if isModuleService(module):
        serviceId = findServiceIdForModule(data, module)
        return "ServiceId {}".format(serviceId)
    return None


## 1. Reserved bandwidth for the different services at the different nodes

In [18]:
# read the omnetpp csv results into a panda data frame
reservation = parseCSV('reservedBandwidth.csv')
reservation.head()   

,run,type,module,name,attrname,attrvalue,value,vectime,vecvalue
0,WithNetworkCalculusCBS_CT_Unicast-0-20231121-1...,runattr,NaN,NaN,configname,WithNetworkCalculusCBS_CT_Unicast,NaN,None,None
1,WithNetworkCalculusCBS_CT_Unicast-0-20231121-1...,runattr,NaN,NaN,datetime,20231121-11:56:18,NaN,None,None
2,WithNetworkCalculusCBS_CT_Unicast-0-20231121-1...,runattr,NaN,NaN,experiment,WithNetworkCalculusCBS_CT_Unicast,NaN,None,None
3,WithNetworkCalculusCBS_CT_Unicast-0-20231121-1...,runattr,NaN,NaN,inifile,omnetpp.ini,NaN,None,None
4,WithNetworkCalculusCBS_CT_Unicast-0-20231121-1...,runattr,NaN,NaN,iterationvars,None,NaN,None,None


In [19]:
# iterate over the modules find the switches (S[*]) and print their reserved bandwidth
runCBSReservation = dict()
for i in range(0, len(reservation)):
    if moduleIdxIsNaN(reservation, i):
        continue
    if isModuleSwitchPort(reservation['module'][i]):
        if isStatReservedBandwidth(reservation['name'][i]):
            # filter reservation['vecvalue'][i] for non zero values and print them
            if not idxHasVecVal(reservation, i):
                continue
            for j in range(0, len(reservation['vecvalue'][i])):
                val, time = getVecValAndTime(reservation, i, j)
                if val != 0:
                    # insert values into switchBandwidth dict
                    runName = getRunName(reservation['run'][i])
                    if runName not in runCBSReservation:
                        runCBSReservation[runName] = dict()
                    switchName = parseModuleNameSwitchCBS(reservation['module'][i])
                    if switchName not in runCBSReservation[runName]:
                        runCBSReservation[runName][switchName] = dict()
                    runCBSReservation[runName][switchName][time] = val


In [20]:
# output the switchBandwidth dict to csv
for run in runCBSReservation.keys():
    switchCBSBandwidthDF = pd.DataFrame.from_dict(runCBSReservation[run])
    # sord inner dict by key
    switchCBSBandwidthDF = switchCBSBandwidthDF.apply(lambda x: x.sort_index())
    switchCBSBandwidthDF.to_csv("export/bandwidth"+run+".csv", index=True, header=True, sep=';')

## 2. Latencies of the different services at different receivers

In [21]:
# read the omnetpp csv results into a panda data frame
latencies = parseCSV('serviceLatencies.csv')
latencies.head()   


,run,type,module,name,attrname,attrvalue,value,vectime,vecvalue
0,WithNetworkCalculusCBS_CT_Unicast-0-20231121-1...,runattr,NaN,NaN,configname,WithNetworkCalculusCBS_CT_Unicast,NaN,None,None
1,WithNetworkCalculusCBS_CT_Unicast-0-20231121-1...,runattr,NaN,NaN,datetime,20231121-11:56:18,NaN,None,None
2,WithNetworkCalculusCBS_CT_Unicast-0-20231121-1...,runattr,NaN,NaN,experiment,WithNetworkCalculusCBS_CT_Unicast,NaN,None,None
3,WithNetworkCalculusCBS_CT_Unicast-0-20231121-1...,runattr,NaN,NaN,inifile,omnetpp.ini,NaN,None,None
4,WithNetworkCalculusCBS_CT_Unicast-0-20231121-1...,runattr,NaN,NaN,iterationvars,None,NaN,None,None


In [22]:
runLatencies = dict()
for i in range(0, len(latencies)):
    if moduleIdxIsNaN(latencies, i):
        continue
    if isModuleService(latencies['module'][i]):
        if isStatServiceLatency(latencies['name'][i]):
            if not idxHasVecVal(latencies, i):
                continue
            runName = getRunName(latencies['run'][i])
            if runName not in runLatencies:
                runLatencies[runName] = dict()
            node = parseNodeName(latencies['module'][i])
            if node not in runLatencies[runName]:
                runLatencies[runName][node] = dict()
            service = parseServiceName(latencies, latencies['module'][i])
            if service not in runLatencies[runName][node]:
                runLatencies[runName][node][service] = dict()
            for j in range(0, len(latencies['vecvalue'][i])):
                val, time = getVecValAndTime(latencies, i, j)
                runLatencies[runName][node][service][time] = val

In [23]:
# plot the latencies one graph per run containing all nodes and services
for run in runLatencies.keys():
    fig, ax = plt.subplots()
    for node in runLatencies[run].keys():
        for service in runLatencies[run][node].keys():
            x = runLatencies[run][node][service].keys()
            y = runLatencies[run][node][service].values()
            ax.plot(x, y, label=node + " " + service)
    ax.legend()
    ax.set_xlabel('time')
    ax.set_ylabel('latency [s]')
    ax.set_title('Latencies for run ' + run)
    ax.grid(True)
    ax.set_xlim(0, 0.1)  # set x-axis limits to 0-100ms
    ax.set_ylim(0, 0.001)  # show latencies below 1ms
    fig.savefig("export/latencies"+run+".png", dpi=300, bbox_inches='tight')
    plt.close(fig)

In [24]:
# plot the latencies one graph per service containing all nodes for all runs
tmpFigs = dict()
tmpAxs = dict()
for run in runLatencies.keys():
    for node in runLatencies[run].keys():
        for service in runLatencies[run][node].keys():
            key = service + " " + node
            if key not in tmpFigs:
                tmpFigs[key], tmpAxs[key] = plt.subplots()
            x = runLatencies[run][node][service].keys()
            y = runLatencies[run][node][service].values()
            ax.plot(x, y, label=node + " " + service)
            tmpAxs[key].plot(x, y, label=run)
for name in tmpFigs.keys():
    tmpAxs[name].legend()
    tmpAxs[name].set_xlabel('time')
    tmpAxs[name].set_ylabel('latency [s]')
    tmpAxs[name].set_title('Latencies for ' + name)
    tmpAxs[name].grid(True)
    tmpAxs[name].set_xlim(0, 0.1)  # set 
    tmpAxs[name].set_ylim(0, 0.001)  # show latencies below 1ms
    tmpFigs[name].savefig("export/latencies"+name+".png", dpi=300, bbox_inches='tight')
    plt.close(tmpFigs[name])

## 3. Queue lengths of the different priorities at the different nodes

In [25]:
# read the omnetpp csv results into a panda data frame
queues = parseCSV('allQueueLengths.csv')
queues.head()

,run,type,module,name,attrname,attrvalue,value,vectime,vecvalue
0,WithNetworkCalculusCBS_CT_Unicast-0-20231121-1...,runattr,NaN,NaN,configname,WithNetworkCalculusCBS_CT_Unicast,NaN,None,None
1,WithNetworkCalculusCBS_CT_Unicast-0-20231121-1...,runattr,NaN,NaN,datetime,20231121-11:56:18,NaN,None,None
2,WithNetworkCalculusCBS_CT_Unicast-0-20231121-1...,runattr,NaN,NaN,experiment,WithNetworkCalculusCBS_CT_Unicast,NaN,None,None
3,WithNetworkCalculusCBS_CT_Unicast-0-20231121-1...,runattr,NaN,NaN,inifile,omnetpp.ini,NaN,None,None
4,WithNetworkCalculusCBS_CT_Unicast-0-20231121-1...,runattr,NaN,NaN,iterationvars,None,NaN,None,None


In [26]:
# determine min max and average queue lengths for each run, node port and queue
runQueueLengths = dict()
for i in range(0, len(queues)):
    if moduleIdxIsNaN(queues, i):
        continue
    moduleName = parseModuleNameQueue(queues['module'][i])
    if moduleName is None or not idxHasVecVal(queues, i):
        continue
    runName = getRunName(queues['run'][i])
    if runName not in runQueueLengths:
        runQueueLengths[runName] = dict()
    if moduleName not in runQueueLengths[runName]:
        runQueueLengths[runName][moduleName] = dict()
    runQueueLengths[runName][moduleName]['min'] = min(queues['vecvalue'][i])
    runQueueLengths[runName][moduleName]['max'] = max(queues['vecvalue'][i])
    runQueueLengths[runName][moduleName]['avg'] = sum(queues['vecvalue'][i])/len(queues['vecvalue'][i])

In [27]:
# plot the min max avg queue lengths as boxes one graph per run containing all nodes and services
for run in runQueueLengths.keys():
    fig, ax = plt.subplots()
    for node in runQueueLengths[run].keys():
        ax.plot([node, node], [runQueueLengths[run][node]['min'], runQueueLengths[run][node]['max']], solid_capstyle="butt")
        ax.plot(node, [runQueueLengths[run][node]['avg']], marker="o")
    ax.set_ylabel('queue length [#packets]')
    ax.set_title('Queue lengths for run ' + run)
    ax.grid(True)
    ax.tick_params(axis='x', rotation=90)
    fig.savefig("export/queues"+run+".png", dpi=300, bbox_inches='tight')
    plt.close(fig)

In [28]:
# pot the min max avg queue lengths as boxes one graph per node containing all runs
tmpFigs = dict()
tmpAxs = dict()
for run in runQueueLengths.keys():
    for node in runQueueLengths[run].keys():
        key = node
        if key not in tmpFigs:
            tmpFigs[key], tmpAxs[key] = plt.subplots()
        tmpAxs[key].plot([run, run], [runQueueLengths[run][node]['min'], runQueueLengths[run][node]['max']], solid_capstyle="butt")
        tmpAxs[key].plot(run, [runQueueLengths[run][node]['avg']], marker="o")
for name in tmpFigs.keys():
    tmpAxs[name].set_ylabel('queue length [#packets]')
    tmpAxs[name].set_title('Queue lengths for ' + name)
    tmpAxs[name].grid(True)
    tmpAxs[name].tick_params(axis='x', rotation=90)
    tmpFigs[name].savefig("export/queues"+name+".png", dpi=300, bbox_inches='tight')
    plt.close(tmpFigs[name])    

C:\Users\Timo\AppData\Local\Temp\ipykernel_10016\1317402689.py:8: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  tmpFigs[key], tmpAxs[key] = plt.subplots()
